# Analyse des vidéos YouTube du prix 100 jours 2024 du Moovjee

**Objectif :** Le but étant de créer un Top 15 des vidéos qui ont reçu le plus de like.

**Plan :**
- Scrapper les données des vidéos YouTube : Récupérer le titre, nb de vues et de like
- Enregistrer ces données dans un Dataframe à l'aide de la librairie Pandas
- Analyser les données

### 1. On importe les librairies

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

### 2. On crée nos fonctions permettant de formater certaines données

In [4]:
# On vérifie si notre valeur est de type "float" ou non
def is_float(a_string):
    try:
        float(a_string)
        return True
    except ValueError:
        return False

# On transforme le format de vues (et like) de YouTube vers un format de nombre (int)
def views_to_int(txt):
    int_result = txt
    int_result = int_result.replace(' vues', '')
    if 'k' in int_result and ',' in int_result:
        int_result = int_result.replace(',', '').replace(' k', '00')
    elif 'k' in int_result and not ',' in int_result:
        int_result = int_result.replace(' k', '000')
    elif ',' in int_result and not 'k' in int_result:
        int_result = int_result.replace(',', '')
    return int(int_result)
    

***Test n°1 :** On teste la fonction "views_to_int()" si elle fonctionne comme on le souhaite.*

In [5]:
print(views_to_int('1,095 vues'))
print(views_to_int('3,5 k vues'))
print(views_to_int('1 k vues'))
print(views_to_int('1 k'))
print(views_to_int('548 vues'))

1095
3500
1000
1000
548


***Test n°2 :** On teste la fonction "is_float()" si elle fonctionne comme on le souhaite*

In [8]:
print(is_float('2'))
print(is_float('2.3'))
print(is_float('un'))
print(is_float('2.3.2'))

True
True
False
False


### 3. On crée notre bot de scrapping avec Selenium

#### 3.1. On définit notre driver avec Chrome

In [ ]:
driver = webdriver.Chrome()

#### 3.2. On se redirige vers l'url de notre choix

*En l'occurrence ici, c'est la playlist contenant toutes les vidéos du prix 100 jours 2024*

In [5]:
driver.get('https://www.youtube.com/watch?v=0BO-fxIrP9E&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0')

#### 3.3. On enregistre les URLs des vidéos de la playlist

In [6]:
tab_a = driver.find_elements(By.CLASS_NAME, 'ytd-playlist-panel-video-renderer')

tab_link = []
for link in tab_a:
    tab_link.append(link.get_attribute('href'))

t = [l for l in tab_link if l != None]
print(t)
print(len(t))

['https://www.youtube.com/watch?v=0BO-fxIrP9E&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0&index=1&pp=iAQB', 'https://www.youtube.com/watch?v=54_pxwgtA-0&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0&index=2&pp=iAQB', 'https://www.youtube.com/watch?v=5Ht_aSSuxOE&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0&index=3&pp=iAQB', 'https://www.youtube.com/watch?v=8HkhOLbaIO8&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0&index=4&pp=iAQB', 'https://www.youtube.com/watch?v=FjnRH5nQxRU&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0&index=5&pp=iAQB', 'https://www.youtube.com/watch?v=FpXpEKyU9vU&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0&index=6&pp=iAQB', 'https://www.youtube.com/watch?v=HgZFTPl-VHg&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0&index=7&pp=iAQB', 'https://www.youtube.com/watch?v=HkMuspSv_LU&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0&index=8&pp=iAQB', 'https://www.youtube.com/watch?v=JKp94NMEhPM&list=PLXwku06I04tLbhEZmTm2cUud5QWxBOSx0&index=9&pp=iAQB', 'https://www.youtube.com/watch?v=KdMkKo8jHy8&list=PLXwku06I04tLbhEZmTm2c

#### 3.4. On récupère les titres, nb de vues et de like, et on enregistre dans un Dataframe

In [7]:
df = pd.DataFrame({'title': [], 'Views': [], 'Likes': []})
for l in t:
    driver.get(l)
    time.sleep(2)
    title = driver.find_element(By.XPATH, '//*[@id="title"]/h1/yt-formatted-string').text
    nb_like = views_to_int(driver.find_element(By.XPATH, '//*[@id="top-level-buttons-computed"]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]').text)
    nb_views = views_to_int(driver.find_element(By.XPATH, '//*[@id="info"]/span[1]').text)

    df.loc[len(df)] = [title, nb_views, nb_like]

### 4. On analyse nos données avec pandas

#### 4.1. On affiche les 10 premières lignes pour vérifier grossièrment s'il les données sont bien enregistrées

In [8]:
df.head(10)

,title,Views,Likes
0,SKANERS (Île-de-France)_ Stéphane Monnel,690,6
1,Siage (Pays de La Loire)_ Hugo Marquet,510,59
2,YESHY (Île-de-France)_Minas Kedin,275,4
3,WASTINA (Normandie)_Précieux Loukoutode,1300,496
4,Stellaire Design (Pays de la Loire)_ Estella R...,484,85
5,Sardorial Wardrobe (Normandie)_ Dorian Monaco,616,186
6,OL DOG STORE (Sud)_Louise Dudoignon,2700,527
7,Spring Fest (Grand Est)_ Clément Lambert,213,3
8,Wecaps (Hauts-de-France)_Adrien Bauw,456,60
9,Solis (Auvergne-Rhône-Alpes)_ Soltane Lhadi,206,24


#### 4.2. On affiche les informations de notre Dataframe
*On vérifie les nb de lignes et colonnes, les types de colonnes, etc.*

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 181 entries, 0 to 180
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   181 non-null    object
 1   Views   181 non-null    int64 
 2   Likes   181 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 5.7+ KB


#### 4.3. On affiche une description générales des données

**Interprétation :** 
- **Nb de vidéos :** 181
- **Nb moyen de vues :** 612 vues
- **Nb moyen de likes :** 125 likes
- **Nb min de vues :** 86 vues
- **Nb max de vues :** 3 800 vues (approximatif)
- **Nb min de likes :** 0 like
- **Nb max de likes :** 1 000 likes (approximatif)

In [10]:
df.describe(include='all')

,title,Views,Likes
count,181,181.000000,181.000000
unique,181,NaN,NaN
top,SKANERS (Île-de-France)_ Stéphane Monnel,NaN,NaN
freq,1,NaN,NaN
mean,NaN,612.204420,125.270718
std,NaN,689.193762,204.667152
min,NaN,86.000000,0.000000
25%,NaN,133.000000,4.000000
50%,NaN,340.000000,41.000000
75%,NaN,841.000000,142.000000


#### 4.4. On crée un Top 15 des vidéos les plus likés

In [25]:
df.sort_values(by=['Likes', 'Views'], ascending=False).head(15)

,title,Views,Likes
132,Collecte D'urine By L (Bretagne)_ Louison Jannee,3800,1000
41,Pensezy (Bretagne)_Léo Saillenfest,2900,946
97,Hairshine (Occitanie)_Falah Nour,1800,797
149,Alguera (Centre-Val de Loire)_Orphé Vezier,1800,759
93,Hypocaps (Nouvelle-Aquitaine)_Maë Legardeur,2400,736
79,Myloot (Île-de-France)_ Robin Thomas,1400,724
86,MonBisouEcolo (Île-De-France)_ Elsie Ahouandjinou,856,698
138,Equipier (Bretagne)_Camille Roger,3100,689
36,Pupil (Hauts-de-France)_Augustin Bellet,2300,642
49,Pictalis (Île-de-France)_Matys Gutierrez y Diez,1700,619


*Les données sont du 27/03/2024 - Analyse à but expérimentale*